<a href="https://colab.research.google.com/github/zacharywendholt/CSCI-B455-Final/blob/master/pokemonBattle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import sklearn
from numpy import loadtxt
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Activation, Flatten
from keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

pokemonURL = 'https://raw.githubusercontent.com/zacharywendholt/CSCI-B455-Final/master/pokemon-challenge/pokemon.csv'
pokemonData = pd.read_csv(pokemonURL)


In [0]:
#re-assigning names to these columns because they have spaces in their name
pokemonData.rename(columns={"Type 1": "Type1"}, inplace=True)
pokemonData.rename(columns={"Type 2": "Type2"}, inplace=True)
# order of legendary pokemon is not randomized. Need to make sure we randomize 
# when we test we need to make sure there is a mix of both legendary and not legendary
cols = pokemonData.columns[2:len(pokemonData.columns)-1]

#changing the string values for type to integer values
le = preprocessing.LabelEncoder()
pokemonData.Type1 = le.fit_transform(pokemonData.Type1)

#type 2 has null values so fill them with na
pokemonData.Type2 = pokemonData.Type2.fillna(method= "ffill")
pokemonData.Type2 = le.fit_transform(pokemonData.Type2)

x = pokemonData.loc[:, cols]
y = pokemonData.Legendary

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size =0.33, random_state=42)



In [0]:
model = Sequential()
model.add(Dense(15, input_dim = len(cols), activation='softmax'))
model.add(Dense(1, input_dim = 15, activation = 'relu'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam',metrics =['accuracy'])

In [0]:
model.fit(x_train,y_train, epochs=10, batch_size = 32, validation_data= (x_test, y_test))

_, accuracy = model.evaluate(x_test,y_test)

print("Accuracy: ",accuracy)
